# FFNN: Single layer example 

In [1]:
def init(dims):
    # We initalize the weights in this case I use where there are a input layer x layer z and output y,
    # so z+y weights to initalize. The initalized are scale for input, so its variance scaled correctly.
    
    # INPUT: Dimesnion of the single layer network
    # OUTPUT: Initalized weight matrix, with each column is a weight fx x->z.
    
    x,z,y=dims
    w = np.sqrt(1/(x+1))
    W = []
    W.append(w*np.random.randn(x+1,z))
    w = np.sqrt(1/(z+1))
    W.append(w*np.random.randn(z+1,y))
    return W

def forward(X, W, n):
    # INPUT: X: Training data, W: Weight matrix, n: points used for training. 
    # OUTPUT: y: Outbut of FFNN, h: a, activation for hidden layer.  
    
    # A fully connected neural network with a_1=max(0,z) (ReLU) as acitvation func and y_out=a_out=softmax(y)
    
    # Look only at subsection of data ( a bit too simple, but for example sake)
    x = X[:,0:n]
    x.shape
    x = np.vstack((x, np.ones((1,n))))
    x.shape

    # % FFNN
    # First layer
    z = W[0].T@x
    h = np.maximum(0, z)
    
    # Second layer
    y_hat = W[1].T@np.vstack((h, np.ones((1,n))))
    y = np.exp(y_hat)/(np.exp(y_hat).sum(axis=0))
    
    return y, h

def backward(X, T, W, n,lr):
    # INPUT: X: Data, T: training sta, W: weights, n: nr trainings points, lr: Learning rate
    
    # Perfom foreward 
    y, h = forward(X, W, n)
    
    
    x = X[:,0:n]
    x.shape
    x = np.vstack((x, np.ones((1,n))))
    x.shape

    delta2=y-T[:,0:n] # Very simple error function of error = model - true = delta2(y)
    Q2=np.vstack((h, np.ones((1,n))))@delta2.T  # dE/d W^2 = h * delta_2 
    W_hat=W[1]                                  # Weight from input layer W^1
    W_hat=W_hat[:-1,:]                          # Remove bias weight from W^1
    a_m=np.where(h > 0, 1, 0)                   # Defin the derivativ of ReLU a'=ReLu'
    delta1=a_m*(W_hat@delta2)                   # delta_1=a'*(W^1*delta_2)
    Q1=x@delta1.T                               # dE/dW^1 = x * delta_1
    
    #Gradient Decent 
    W[0]=W[0]-lr*Q1
    W[1]=W[1]-lr*Q2
    print(f'loop {i}')
    return W

def train(X, T, dims, epochs, lr, batchsize=1):

    # Train the 2-layer network using mini-batch gradient descent.
    
    # INPUT: X : Training input data, T : Target, dims : (x, z, y) layer size, epochs : Number of epochs
    #        lr : Learning rate, batchsize :  Size of mini-batch
            
    # OUTPUT: W : Trained weights,  losses :Loss at each epoch
 
    
    rg = np.random.default_rng()
    W = init(dims)
    losses = []
    m = X.shape[1]
    
    for e in range(epochs):
        order = rg.permutation(m)
        epoch_loss = 0
        
        for k in range(0, m, batchsize):
            batch = order[k:k + batchsize]
            X_batch = X[:, batch]
            T_batch = T[:, batch]
            W, loss = backward(X_batch, T_batch, W, n=batchsize, lr=lr)
            epoch_loss += loss
            
        losses.append(epoch_loss)
        print(f"Epoch {e+1}, loss {epoch_loss:.4f}")
    
    return W, losses

# FFNN: Multilayer 

In [2]:
import numpy as np

rg = np.random.default_rng() # random munber generator 

def init(dims):
    """
    Initialize weights for a multi-layer feedforward neural network.
    
    INPUT:
        dims : Layer dimensions [input, hidden1, hidden2, ..., output]
    
    OUTPUT:
        W : List of weight matrices for each layer.
            Each W[l] has shape (dims[l]+1, dims[l+1]) and includes bias weights.
    """
    
    W = []
    for l in range(len(dims) - 1):
        
        # initialization for ReLU layers: variance = 2 / fan_in, makes sure variance is stable for relu 
        fan_in = dims[l] + 1
        scale = np.sqrt(2 / fan_in)
        W.append(scale * rg.standard_normal(size=(fan_in, dims[l+1])))
        
    return W


def forward(X, W):
    """
    Perform the forward pass through all layers.
    
    INPUT:
        X : Input data (shape: features × samples)
        W : Weight matrices for each layer
    
    OUTPUT:
        y : Output of forward with softmax aplied
        h : List containing activations (hidden layers and final output pre-softmax)
    """
    
    h = []  # store activations of each layer
    a = X   # input layer
    
    # Forward through hidden layers
    for l in range(len(W) - 1):
        # Add bias term
        a = np.vstack((a, np.ones((1, a.shape[1]))))
        
        # Linear transform
        z = W[l].T @ a
        
        # ReLU activation
        a = np.maximum(0, z)
        h.append(a)
    
    # Output layer (no ReLU, only linear before softmax)
    a = np.vstack((a, np.ones((1, a.shape[1]))))
    y_hat = W[-1].T @ a
    h.append(y_hat)
    
    # Last element in h is pre-softmax output
    y_hat = h[-1]
    
    # Numerically stable softmax
    exp_y = np.exp(y_hat - np.max(y_hat, axis=0))
    y = exp_y / exp_y.sum(axis=0)
    
    return y, h


def backward(X, T, W, lr):
    """
    Perform one backward pass and update weights.
    
    INPUT:
        X : Input data (features × samples)
        T : Target labels (one-hot encoded)
        W : Current weight matrices
        lr : Learning rate
    
    OUTPUT:
        W : Updated weight matrices
        loss : Total loss for this batch
    """
    
    n = X.shape[1]
    y, h = forward(X, W)

    # --- Loss ---
    loss = -np.sum(T * np.log(y + 1e-15)) # cross-entropy loss for multi-class classification
    
    # --- Backpropagation --- (Like in example above just more layers)
    delta = y - T  # output layer error (simple example)
    
    # --- First layer update ---
    x_bias = np.vstack((X, np.ones((1, n))))
    Q0 = x_bias @ delta.T
    W[0] -= (lr / n) * Q0
    
    # --- Update all other layers
    
    for l in range(len(W) - 1, 0, -1):
        # Get hidden activation for this layer
        h_l = h[l-1] if l-1 >= 0 else X
        
        # Add bias term
        h_l = np.vstack((h_l, np.ones((1, h_l.shape[1]))))
        
        # Compute gradient for this layer
        Q = h_l @ delta.T
        
        # Update weights
        W[l] -= (lr / n) * Q
        
        # Backpropagate error to previous layer
        W_hat = W[l][:-1, :]  # remove bias weights
        
        if l > 0: # compute delta for next layer
            a_prime = (h[l-1] > 0).astype(float)  # ReLU derivative
            delta = a_prime * (W_hat @ delta)
    
    return W, loss


def train(X, T, W, epochs, lr, batchsize=1):
    """
    Train the multi-layer neural network using gradient descent.
    
    INPUT:
        X : Input data (features × samples)
        T : Target one-hot labels (classes × samples)
        W : Initialized weight matrices
        epochs :  Number of training epochs
        lr : Learning rate
        batchsize : Size of each training batch
    
    OUTPUT:
        W : Trained weight matrices
        losses : Total loss for each epoch
    """
    
    m = X.shape[1]
    losses = []
    
    for e in range(epochs):
        order = rg.permutation(m) # mini-batch is drawn in a random order
        epoch_loss = 0
        
        for k in range(0, m, batchsize):
            batch = order[k:k+batchsize]
            X_batch = X[:, batch]
            T_batch = T[:, batch]
            
            W, loss = backward(X_batch, T_batch, W, lr)
            epoch_loss += loss
        
        losses.append(epoch_loss)
        print(f"Epoch {e+1}, loss {epoch_loss:.4f}")
    
    return W, losses


# Working w. MNIST minimal change 


In [5]:
### 1. Import Required Libraries

import numpy as np
from keras.datasets import mnist
from keras.utils import to_categorical
import tensorflow

def softmax(y_hat):
    y_hat = y_hat - np.max(y_hat, axis=0, keepdims=True)  # prevent overflow
    exp_scores = np.exp(y_hat)
    return exp_scores / np.sum(exp_scores, axis=0, keepdims=True)


### 2. Load MNIST Data

# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Reshape and normalize inputs 
X_train = X_train.reshape(-1, 28*28) / 255.0
X_test = X_test.reshape(-1, 28*28) / 255.0

# One-hot encode labels
T_train = to_categorical(y_train, num_classes=10)
T_test = to_categorical(y_test, num_classes=10)


### 3. Initialize Network Parameters

def init(dims):
    W = []
    for i in range(len(dims) - 1):
        W.append(np.random.randn(dims[i] + 1, dims[i + 1]) * np.sqrt(2 / (dims[i])))
    return W

dims = [784, 32, 32, 10]  # Input layer (784), hidden layer (128), output layer (10)
W = init(dims)


### 4. Define Forward Pass

def forward(X, W):
    h = []
    a = X
    for l in range(len(W) - 1):
        a = np.vstack([a, np.ones(a.shape[1])])  # Add bias term
        z = W[l].T @ a
        a = np.maximum(0, z)  # ReLU activation
        h.append(a)
    a = np.vstack([a, np.ones(a.shape[1])])  # Add bias term
    y_hat = W[-1].T @ a
    y = softmax(y_hat)  # new stable version
  # Softmax
    return y, h


### 5. Define Backward Pass

def backward(X, T, W, h, eta):
    m = X.shape[1]
    y, _ = forward(X, W)
    delta = y - T
    for l in range(len(W) - 1, 0, -1):
        a_prev = np.vstack([h[l-1], np.ones(h[l-1].shape[1])])  # Add bias term
        Q = a_prev @ delta.T
        W[l] -= (eta / m) * Q
        delta = W[l][:-1, :] @ delta
        delta *= h[l-1] > 0  # ReLU derivative
    a_prev = np.vstack([X, np.ones(X.shape[1])])  # Add bias term
    Q = a_prev @ delta.T
    W[0] -= eta * Q
    epsilon = 1e-12
    loss = -np.sum(np.log(np.sum(y * T, axis=0) + epsilon))
    return W, loss


### 6. Training Loop

def train(X, T, W, epochs, eta, batchsize=32):
    m = X.shape[1]
    losses = []
    for epoch in range(epochs):
        order = np.random.permutation(m)
        epoch_loss = 0
        for i in range(0, m, batchsize):
            batch = order[i:i+batchsize]
            X_batch = X[:, batch]
            T_batch = T[:, batch]
            _, h = forward(X_batch, W)
            W, loss = backward(X_batch, T_batch, W, h, eta)
            epoch_loss += loss
        losses.append(epoch_loss)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")
    return W, losses


### 7. Train the Model

epochs = 100
eta = 0.001
W, losses = train(X_train.T, T_train.T, W, epochs, eta)

### 8. Evaluate the Model

def predict(X, W):
    y, _ = forward(X, W)
    return np.argmax(y, axis=0)

y_pred = predict(X_test.T, W)
accuracy = np.mean(y_pred == y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Epoch 1/100, Loss: 37932.0995
Epoch 2/100, Loss: 18044.1915
Epoch 3/100, Loss: 15187.7958
Epoch 4/100, Loss: 13638.4732
Epoch 5/100, Loss: 12602.7522
Epoch 6/100, Loss: 11802.6428
Epoch 7/100, Loss: 11152.7841
Epoch 8/100, Loss: 10658.2346
Epoch 9/100, Loss: 10195.0382
Epoch 10/100, Loss: 9813.7084
Epoch 11/100, Loss: 9456.8015
Epoch 12/100, Loss: 9180.1242
Epoch 13/100, Loss: 8899.8566
Epoch 14/100, Loss: 8653.8921
Epoch 15/100, Loss: 8405.4606
Epoch 16/100, Loss: 8185.0095
Epoch 17/100, Loss: 7977.0305
Epoch 18/100, Loss: 7784.6646
Epoch 19/100, Loss: 7602.3839
Epoch 20/100, Loss: 7453.3020
Epoch 21/100, Loss: 7295.7094
Epoch 22/100, Loss: 7148.7067
Epoch 23/100, Loss: 6982.9027
Epoch 24/100, Loss: 6854.6996
Epoch 25/100, Loss: 6735.9956
Epoch 26/100, Loss: 6601.0594
Epoch 27/100, Loss: 6477.2278
Epoch 28/100, Loss: 6373.1218
Epoch 29/100, Loss: 6259.8047
Epoch 30/100, Loss: 6130.8877
Epoch 31/100, Loss: 6037.4217
Epoch 32/100, Loss: 5947.5378
Epoch 33/100, Loss: 5841.4754
Epoch 34/1